# Sprawozdanie z laboratorium 3

***Autor: Adam Dąbkowski***

Celem trzeciego laboratorium jest zaimplementowanie algorytmu ***minimax*** z obcinaniem $\alpha - \beta$, który różnych ruchów o tej samej jakości zwraca losowy z nich. Dodatkowo należy wykorzystać implementację, aby porównać wpływ doboru głębokości przeszukiwania dla gry Pick.

### 0. Importowanie niezbędnych bibliotek i klas

In [2]:
import itertools
from typing import Iterable, List, Optional
from Lab3.two_player_games.game import Game
from Lab3.two_player_games.move import Move
from Lab3.two_player_games.player import Player
from Lab3.two_player_games.state import State

from copy import deepcopy
import random

### 1. Implementacja

Aby algorytm ***minimax*** mógł wybrać ruch, który znacznie przybliży gracza do zwycięstwa, konieczne jest określenie jakości danego ruchu. W tym celu konieczne jest zaimplementowanie funkcji heurystycznej, która na podstawie danego stanu gry określi, czy dobór liczb, jest dobry, bądź nie.

Zanim skorzystamy z funkcji heurystycznej, konieczne jest najpierw przekształcenie klasy ***Pick***. Chodzi tu mianowicie o wyznaczenie za pomocą metody **get_game_combinations()** wszystkich możliwych kombinacji liczb w danej grze, które będzie reprezentowane przez ***self.game_combinations***.

In [21]:
class Pick(Game):
    """Class that represents the Pick game"""
    FIRST_PLAYER_DEFAULT_CHAR = '1'
    SECOND_PLAYER_DEFAULT_CHAR = '2'

    def __init__(self, first_player: Player = None, second_player: Player = None, n: int = 4):
        """
        Initializes game.

        Parameters:
            first_player: the player that will go first (if None is passed, a player will be created)
            second_player: the player that will go second (if None is passed, a player will be created)
            n: the subset size of picked numbers that should sum up to aim value
        """
        self.first_player = first_player or Player(self.FIRST_PLAYER_DEFAULT_CHAR)
        self.second_player = second_player or Player(self.SECOND_PLAYER_DEFAULT_CHAR)

        state = PickState(self.first_player, self.second_player, n)

        super().__init__(state)

        self.game_combinations = self.get_game_combinations()

    def get_game_combinations(self):
        potential_numbers = range(1, self.state.max_number + 1)

        def backtrack(start, aim_value):
            combinations = []
            for index in range(start, len(potential_numbers)):
                number = potential_numbers[index]
                if index > start and number == potential_numbers[index - 1]:
                    continue
                if number < aim_value:
                    combinations.extend([number] + comb for comb in backtrack(index + 1, aim_value - number))
                else:
                    if number == aim_value:
                        combinations.append([number])
                    break
            return combinations

        game_combinations = backtrack(0, self.state.aim_value)
        game_combinations = [combination for combination in game_combinations if len(combination) == self.state.n]

        return game_combinations


class PickMove(Move):
    """
    Class that represents a move in the PickMove game

    Variables:
        number: selected number (from 1 to n^2)
    """

    def __init__(self, number: int):
        self.number = number

    def __eq__(self, o: object) -> bool:
        if not isinstance(o, PickMove):
            return False
        return self.number == o.number

    def __str__(self):
        return str(self.number)


class PickState(State):
    """Class that represents a state in the PickState game"""

    def __init__(self,
                 current_player: Player, other_player: Player, n,
                 current_player_numbers: List[int] = None,
                 other_player_numbers: List[int] = None):
        """Creates the state. Do not call directly."""

        if current_player_numbers is None:
            current_player_numbers = []
        if other_player_numbers is None:
            other_player_numbers = []

        self.current_player_numbers = current_player_numbers
        self.other_player_numbers = other_player_numbers
        self.selected_numbers = set(self.current_player_numbers).union(self.other_player_numbers)
        self.n = n
        self.max_number = n ** 2
        self.aim_value = int((n ** 2 * (n ** 2 + 1)) / (2 * n))
        super().__init__(current_player, other_player)

    def get_moves(self) -> Iterable[PickMove]:
        return [PickMove(number) for number in range(1, self.max_number + 1) if number not in self.selected_numbers]

    def make_move(self, move: PickMove) -> 'PickState':
        if move.number > self.max_number or move.number in self.selected_numbers:
            raise ValueError("Invalid move")
        else:
            next_player = self._other_player
            next_player_numbers = self.other_player_numbers

            other_player = self._current_player
            other_player_numbers = self.current_player_numbers + [move.number]

        return PickState(
            next_player, other_player, self.n, next_player_numbers, other_player_numbers
        )

    def is_finished(self) -> bool:
        return self._check_if_sums_to_aim_value(self.current_player_numbers) or \
               self._check_if_sums_to_aim_value(self.other_player_numbers) or \
               len(self.selected_numbers) == self.max_number

    def get_winner(self) -> Optional[Player]:
        if not self.is_finished():
            return None
        if self._check_if_sums_to_aim_value(self.current_player_numbers):
            return self._current_player
        elif self._check_if_sums_to_aim_value(self.other_player_numbers):
            return self._other_player
        else:
            return None

    def __str__(self) -> str:
        return f"n: {self.n}, aim_value: {self.aim_value}" \
               f"\nCurrent player: {self._current_player.char}, Numbers: " \
               f"{'[]' if not self.current_player_numbers else sorted(self.current_player_numbers)}," \
               f"\nOther player: {self._other_player.char}, Numbers: " \
               f"{'[]' if not self.other_player_numbers else sorted(self.other_player_numbers)}"

    # below are helper methods for the public interface

    def _check_if_sums_to_aim_value(self, numbers: List[int]) -> bool:
        return self.aim_value in [sum(i) for i in itertools.combinations(numbers, self.n)]

Mając wyznaczone kombinacje liczb, możemy przejść do implementacji funkcji heurystycznej **get_heuristic_score()**. Funkcja ta na podstawie wykonanych już ruchów gracza, a także potencjalnych ruchów, które gracz jest w stanie wykonać, wyznacza heurystyczny wynik wskazujący na częstość występowania danych liczb w danych kombinacjach. Im wynik wyższy, tym lepiej. Jeżeli zdarzy się sytuacja, w której wewnątrz kombinacji znajdzie się zbiór, który sumuje się do oczekiwanej wartości, zwracany jest wynik ***-100000*** lub ***100000*** w zależności od tego, czy mamy do czynienia z ***max_player***, czy nie.

In [22]:
def get_heuristic_score(game, max_player):
    if not game.is_finished():
        heuristic_score = 0
        potential_combinations = []

        player_moves = game.state.current_player_numbers
        potential_moves = [item.number for item in game.state.get_moves()]

        for combination in game.game_combinations:
            invalid_combination = False
            for item in combination:
                invalid_combination = True if item not in potential_moves and item not in player_moves else False
                if invalid_combination:
                    break
            if not invalid_combination:
                potential_combinations.append(combination)

        for combination in potential_combinations:
            combination_score = 0
            for number in combination:
                combination_score += 1 if number in player_moves else 0

            heuristic_score += combination_score

        return -heuristic_score if max_player else heuristic_score
    else:
        if game.get_winner():
            return -100000 if max_player else 100000
        else:
            return 0

Mając tak zaimplementowaną funkcję heurystyczną, jesteśmy już w stanie skorzystać z algorytmu ***minimax***, z obcinaniem $\alpha - \beta$. Algorytm ten jest zgodny z wymaganiem postawionym w poleceniu, a mianowicie dla różnych ruchów o tej samej jakości zwraca losowy z nich.

In [23]:
def minimax_alpha_beta(game, depth, alpha, beta, max_player):
    if depth == 0 or game.state.is_finished():
        return [get_heuristic_score(game, max_player), 0]

    if max_player:
        max_value = float('-inf')
        best_moves = {}

        for move in game.get_moves():
            current_game = deepcopy(game)
            current_game.make_move(move)
            [value, _ ] = minimax_alpha_beta(current_game, depth - 1, alpha, beta, False)

            if value > max_value:
                max_value = value
                best_moves = {f"{max_value}": [move]}

            if value == max_value:
                best_moves[f"{max_value}"].append(move)

            if max_value >= beta:
                break

            alpha = max(alpha, max_value)

        return [max_value, random.choice(best_moves[f"{max_value}"])]

    else:
        min_value = float('inf')
        best_moves = {}

        for move in game.get_moves():
            current_game = deepcopy(game)
            current_game.make_move(move)

            [value, _ ] = minimax_alpha_beta(current_game, depth - 1, alpha, beta, True)

            if value < min_value:
                min_value = value
                best_moves = {f"{min_value}": [move]}

            if value == min_value:
                best_moves[f"{min_value}"].append(move)

            if min_value <= alpha:
                break

            beta = min(beta, min_value)

        return [min_value, random.choice(best_moves[f"{min_value}"])]

Aby móc przetestować powyższą implementację, utworzona została funkcja **game_simulation()** , która jako parametry wejściowe przyjmuje ***n*** - rozmiar zadania, ***first_player_depth*** - głębokość przeszukiwania dla pierwszego gracza oraz ***second_player_depth*** - głębokość przeszukiwania dla drugiego gracza.

In [24]:
def game_simulation(n=4, first_player_depth=1, second_player_depth=1):
    game = Pick(n=n)
    turn = 1

    while game.is_finished() is False:
        print(f"Turn {turn} ------------------------------------------------------------------------------------------")

        [heuristic_score, move] = minimax_alpha_beta(game, first_player_depth, float('-inf'), float('inf'), True)
        game.make_move(move)

        print(f"Player {game.state._other_player.char}: selected move = {move.number}, heuristic_score = {heuristic_score}, player moves = {game.state.other_player_numbers}")

        if game.is_finished():
            break

        [heuristic_score, move] = minimax_alpha_beta(game, second_player_depth, float('-inf'), float('inf'), True)
        game.make_move(move)

        print(f"Player {game.state._other_player.char}: selected move = {move.number}, heuristic_score = {heuristic_score}, player moves = {game.state.other_player_numbers}")

        turn += 1

    if game.state.get_winner():
        print(f"Winner = Player {game.state.get_winner().char}")
    else:
        print(f"Draw")

### 2. Symulacja

Symulacja została przeprowadzona dla różnych kombinacji wartości ***first_player_depth*** oraz ***second_player_depth***, tak aby zobaczyć, jak znaczący wpływ ma kolejność wykonywania ruchów oraz głębokość przeszukiwania.

<b>2.1 ***first_player_depth > second_player_depth***</b>

In [19]:
game_simulation(n=4, first_player_depth=2, second_player_depth=1)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 15, heuristic_score = -18, player moves = [15]
Player 2: selected move = 16, heuristic_score = 19, player moves = [16]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 1, heuristic_score = -26, player moves = [15, 1]
Player 2: selected move = 9, heuristic_score = 26, player moves = [16, 9]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 3, heuristic_score = -32, player moves = [15, 1, 3]
Player 2: selected move = 2, heuristic_score = 32, player moves = [16, 9, 2]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 8, heuristic_score = -34, player moves = [15, 1, 3, 8]
Player 2: selected move = 7, heuristic_score = 100000, player moves = [16, 9, 

In [14]:
game_simulation(n=4, first_player_depth=3, second_player_depth=1)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 7, heuristic_score = 14, player moves = [7]
Player 2: selected move = 1, heuristic_score = 20, player moves = [1]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 2, heuristic_score = 21, player moves = [7, 2]
Player 2: selected move = 3, heuristic_score = 33, player moves = [1, 3]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 13, heuristic_score = 27, player moves = [7, 2, 13]
Player 2: selected move = 16, heuristic_score = 39, player moves = [1, 3, 16]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 12, heuristic_score = 100000, player moves = [7, 2, 13, 12]
Winner = Player 1


In [19]:
game_simulation(n=4, first_player_depth=3, second_player_depth=2)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 8, heuristic_score = 14, player moves = [8]
Player 2: selected move = 14, heuristic_score = -13, player moves = [14]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 9, heuristic_score = 23, player moves = [8, 9]
Player 2: selected move = 4, heuristic_score = -23, player moves = [14, 4]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 6, heuristic_score = 27, player moves = [8, 9, 6]
Player 2: selected move = 16, heuristic_score = -27, player moves = [14, 4, 16]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 15, heuristic_score = 100000, player moves = [8, 9, 6, 15]
Player 2: selected move = 12, heuristic_score = -100000, player moves = [14,

In [25]:
game_simulation(n=4, first_player_depth=4, second_player_depth=1)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 4, heuristic_score = -26, player moves = [4]
Player 2: selected move = 1, heuristic_score = 20, player moves = [1]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 16, heuristic_score = -34, player moves = [4, 16]
Player 2: selected move = 14, heuristic_score = 28, player moves = [1, 14]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 11, heuristic_score = -33, player moves = [4, 16, 11]
Player 2: selected move = 15, heuristic_score = 36, player moves = [1, 14, 15]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 2, heuristic_score = 100000, player moves = [4, 16, 11, 2]
Player 2: selected move = 3, heuristic_score = 35, player moves = [1, 14

In [15]:
game_simulation(n=4, first_player_depth=4, second_player_depth=2)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 13, heuristic_score = -26, player moves = [13]
Player 2: selected move = 1, heuristic_score = -13, player moves = [1]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 16, heuristic_score = -35, player moves = [13, 16]
Player 2: selected move = 2, heuristic_score = -16, player moves = [1, 2]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 15, heuristic_score = -29, player moves = [13, 16, 15]
Player 2: selected move = 4, heuristic_score = -14, player moves = [1, 2, 4]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 14, heuristic_score = -18, player moves = [13, 16, 15, 14]
Player 2: selected move = 3, heuristic_score = -7, player moves = [1, 

In [27]:
game_simulation(n=4, first_player_depth=4, second_player_depth=3)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 7, heuristic_score = -26, player moves = [7]
Player 2: selected move = 2, heuristic_score = 29, player moves = [2]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 9, heuristic_score = -34, player moves = [7, 9]
Player 2: selected move = 15, heuristic_score = 37, player moves = [2, 15]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 10, heuristic_score = -36, player moves = [7, 9, 10]
Player 2: selected move = 8, heuristic_score = 36, player moves = [2, 15, 8]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 11, heuristic_score = 100000, player moves = [7, 9, 10, 11]
Player 2: selected move = 16, heuristic_score = -100000, player moves = [2, 

<b>2.2 ***first_player_depth < second_player_depth***</b>

In [14]:
game_simulation(n=4, first_player_depth=1, second_player_depth=2)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 8, heuristic_score = 0, player moves = [8]
Player 2: selected move = 15, heuristic_score = -13, player moves = [15]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 16, heuristic_score = 15, player moves = [8, 16]
Player 2: selected move = 1, heuristic_score = -20, player moves = [15, 1]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 9, heuristic_score = 20, player moves = [8, 16, 9]
Player 2: selected move = 5, heuristic_score = -24, player moves = [15, 1, 5]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 7, heuristic_score = 26, player moves = [8, 16, 9, 7]
Player 2: selected move = 13, heuristic_score = 100000, player moves = [15, 1, 5,

In [7]:
game_simulation(n=4, first_player_depth=1, second_player_depth=3)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 1, heuristic_score = 0, player moves = [1]
Player 2: selected move = 13, heuristic_score = 26, player moves = [13]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 15, heuristic_score = 16, player moves = [1, 15]
Player 2: selected move = 3, heuristic_score = 32, player moves = [13, 3]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 9, heuristic_score = 25, player moves = [1, 15, 9]
Player 2: selected move = 12, heuristic_score = 35, player moves = [13, 3, 12]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 16, heuristic_score = 30, player moves = [1, 15, 9, 16]
Player 2: selected move = 6, heuristic_score = 100000, player moves = [13, 3, 12

In [13]:
game_simulation(n=4, first_player_depth=1, second_player_depth=4)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 4, heuristic_score = 0, player moves = [4]
Player 2: selected move = 13, heuristic_score = -21, player moves = [13]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 15, heuristic_score = 14, player moves = [4, 15]
Player 2: selected move = 11, heuristic_score = -27, player moves = [13, 11]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 16, heuristic_score = 26, player moves = [4, 15, 16]
Player 2: selected move = 7, heuristic_score = -29, player moves = [13, 11, 7]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 1, heuristic_score = 31, player moves = [4, 15, 16, 1]
Player 2: selected move = 3, heuristic_score = 100000, player moves = [13, 

In [13]:
game_simulation(n=4, first_player_depth=2, second_player_depth=3)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 12, heuristic_score = -18, player moves = [12]
Player 2: selected move = 9, heuristic_score = 29, player moves = [9]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 10, heuristic_score = -29, player moves = [12, 10]
Player 2: selected move = 16, heuristic_score = 36, player moves = [9, 16]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 15, heuristic_score = -36, player moves = [12, 10, 15]
Player 2: selected move = 2, heuristic_score = 40, player moves = [9, 16, 2]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 7, heuristic_score = -40, player moves = [12, 10, 15, 7]
Player 2: selected move = 5, heuristic_score = 100000, player moves = [9

In [40]:
game_simulation(n=4, first_player_depth=2, second_player_depth=4)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 1, heuristic_score = -18, player moves = [1]
Player 2: selected move = 6, heuristic_score = -22, player moves = [6]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 16, heuristic_score = -24, player moves = [1, 16]
Player 2: selected move = 15, heuristic_score = -28, player moves = [6, 15]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 2, heuristic_score = -29, player moves = [1, 16, 2]
Player 2: selected move = 3, heuristic_score = -27, player moves = [6, 15, 3]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 12, heuristic_score = -31, player moves = [1, 16, 2, 12]
Player 2: selected move = 10, heuristic_score = 100000, player moves = [6, 

In [14]:
game_simulation(n=4, first_player_depth=2, second_player_depth=4)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 8, heuristic_score = -18, player moves = [8]
Player 2: selected move = 10, heuristic_score = -21, player moves = [10]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 16, heuristic_score = -27, player moves = [8, 16]
Player 2: selected move = 14, heuristic_score = -27, player moves = [10, 14]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 15, heuristic_score = -31, player moves = [8, 16, 15]
Player 2: selected move = 1, heuristic_score = -20, player moves = [10, 14, 1]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 9, heuristic_score = -36, player moves = [8, 16, 15, 9]
Player 2: selected move = 11, heuristic_score = -100000, player moves 

In [31]:
game_simulation(n=4, first_player_depth=3, second_player_depth=4)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 8, heuristic_score = 14, player moves = [8]
Player 2: selected move = 11, heuristic_score = -21, player moves = [11]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 9, heuristic_score = 23, player moves = [8, 9]
Player 2: selected move = 16, heuristic_score = -26, player moves = [11, 16]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 15, heuristic_score = 26, player moves = [8, 9, 15]
Player 2: selected move = 5, heuristic_score = -100000, player moves = [11, 16, 5]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 3, heuristic_score = 100000, player moves = [8, 9, 15, 3]
Player 2: selected move = 2, heuristic_score = 100000, player moves = 

In [30]:
game_simulation(n=4, first_player_depth=5, second_player_depth=6)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 7, heuristic_score = 22, player moves = [7]
Player 2: selected move = 5, heuristic_score = -26, player moves = [5]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 16, heuristic_score = 28, player moves = [7, 16]
Player 2: selected move = 13, heuristic_score = -100000, player moves = [5, 13]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 14, heuristic_score = 100000, player moves = [7, 16, 14]
Player 2: selected move = 15, heuristic_score = 100000, player moves = [5, 13, 15]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 1, heuristic_score = 100000, player moves = [7, 16, 14, 1]
Player 2: selected move = 11, heuristic_score = -100000, play

<b>2.3 ***first_player_depth = second_player_depth***</b>

In [48]:
game_simulation(n=4, first_player_depth=1, second_player_depth=1)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 5, heuristic_score = 0, player moves = [5]
Player 2: selected move = 1, heuristic_score = 20, player moves = [1]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 2, heuristic_score = 16, player moves = [5, 2]
Player 2: selected move = 3, heuristic_score = 34, player moves = [1, 3]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 4, heuristic_score = 27, player moves = [5, 2, 4]
Player 2: selected move = 16, heuristic_score = 41, player moves = [1, 3, 16]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 15, heuristic_score = 28, player moves = [5, 2, 4, 15]
Player 2: selected move = 14, heuristic_score = 100000, player moves = [1, 3, 16, 14]
Wi

In [23]:
game_simulation(n=4, first_player_depth=1, second_player_depth=1)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 2, heuristic_score = 0, player moves = [2]
Player 2: selected move = 1, heuristic_score = 19, player moves = [1]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 3, heuristic_score = 17, player moves = [2, 3]
Player 2: selected move = 4, heuristic_score = 34, player moves = [1, 4]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 5, heuristic_score = 28, player moves = [2, 3, 5]
Player 2: selected move = 9, heuristic_score = 39, player moves = [1, 4, 9]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 12, heuristic_score = 31, player moves = [2, 3, 5, 12]
Player 2: selected move = 8, heuristic_score = 38, player moves = [1, 4, 9, 8]
Turn 5 ----

In [20]:
game_simulation(n=4, first_player_depth=2, second_player_depth=2)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 14, heuristic_score = -18, player moves = [14]
Player 2: selected move = 1, heuristic_score = -13, player moves = [1]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 16, heuristic_score = -26, player moves = [14, 16]
Player 2: selected move = 3, heuristic_score = -16, player moves = [1, 3]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 15, heuristic_score = -23, player moves = [14, 16, 15]
Player 2: selected move = 2, heuristic_score = -12, player moves = [1, 3, 2]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 13, heuristic_score = -18, player moves = [14, 16, 15, 13]
Player 2: selected move = 4, heuristic_score = -7, player moves = [1, 

In [33]:
game_simulation(n=4, first_player_depth=3, second_player_depth=3)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 9, heuristic_score = 14, player moves = [9]
Player 2: selected move = 15, heuristic_score = 29, player moves = [15]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 13, heuristic_score = 22, player moves = [9, 13]
Player 2: selected move = 2, heuristic_score = 36, player moves = [15, 2]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 1, heuristic_score = 27, player moves = [9, 13, 1]
Player 2: selected move = 11, heuristic_score = 35, player moves = [15, 2, 11]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 6, heuristic_score = 25, player moves = [9, 13, 1, 6]
Player 2: selected move = 14, heuristic_score = 100000, player moves = [15, 2, 11

In [25]:
game_simulation(n=4, first_player_depth=4, second_player_depth=4)

Turn 1 ------------------------------------------------------------------------------------------
Player 1: selected move = 9, heuristic_score = -26, player moves = [9]
Player 2: selected move = 13, heuristic_score = -21, player moves = [13]
Turn 2 ------------------------------------------------------------------------------------------
Player 1: selected move = 6, heuristic_score = -33, player moves = [9, 6]
Player 2: selected move = 8, heuristic_score = -25, player moves = [13, 8]
Turn 3 ------------------------------------------------------------------------------------------
Player 1: selected move = 14, heuristic_score = -31, player moves = [9, 6, 14]
Player 2: selected move = 15, heuristic_score = -100000, player moves = [13, 8, 15]
Turn 4 ------------------------------------------------------------------------------------------
Player 1: selected move = 4, heuristic_score = 100000, player moves = [9, 6, 14, 4]
Player 2: selected move = 1, heuristic_score = -100000, player moves

### 3. Podsumowanie

Analizując powyższe symulacje, zauważamy, że w zdecydowanej większości przypadków o tym, kto zwycięży, decyduje głębokość przeszukiwania. Oczywiście zdarza się, że zwycięży gracz cechujący się mniejszą głębokością, jednakże występuje to szczególnie w przypadkach, gdy głębokości obu graczy są do siebie podobne. Innym ważnym czynnikiem jest kolejność wykonywania ruchu. Widzimy to, chociażby dla ***first_player_depth=5***, ***second_player_depth=6***. Pomimo faktu, że drugi gracz ma większą głębokość przeszukiwania, w danej sytuacji wygrywa gracz pierwszy. Wpływ na to ma głównie to, że zarówno wartość ***5***, jak i ***6*** są to wartości bardzo duże, i nawet jeżeli głębokość ***5*** jest mniejsza, to i tak jest na tyle duża, aby umożliwić zwycięstwo graczowi pierwszemu. W przypadku identycznych głębokości przeszukiwania pojedynek jest bardziej wyrównany, jednakże inicjatywę ma zawsze gracz pierwszy i to on wygrywa częściej. Może się tak zdarzyć, że osiągnięty zostanie remis, co widzimy dla głębokości równej ***2***, ale także dla ***first_player_depth=4*** i ***second_player_depth=2***

| First Player Depth | Second Player Depth | Result              |
|--------------------|---------------------|---------------------|
| 1                  | 1                   | Player 1 / Player 2 |
| 2                  | 1                   | Player 2            |
| 3                  | 1                   | Player 1            |
| 4                  | 1                   | Player 1            |
| 2                  | 2                   | Draw                |
| 3                  | 2                   | Player 2            |
| 4                  | 2                   | Draw                |
| 3                  | 3                   | Player 1            |
| 4                  | 3                   | Player 1            |
| 4                  | 4                   | Player 1            |
| 1                  | 2                   | Player 2            |
| 1                  | 3                   | Player 2            |
| 1                  | 4                   | Player 2            |
| 2                  | 3                   | Player 2            |
| 2                  | 4                   | Player 1 / Player 2 |
| 3                  | 4                   | Player 2            |
| 5                  | 6                   | Player 1            |